In [1]:
"""
Add mypy type-checking cell magic to jupyter/ipython.
Save this script to your ipython profile's startup directory.
IPython's directories can be found via `ipython locate [profile]` to find the 
current ipython directory and ipython profile directory, respectively.
For example, this file could exist on a path like this on mac:
/Users/yourusername/.ipython/profile_default/startup/typecheck.py
where /Users/yourusername/.ipython/profile_default/ is the ipython directory for
the default profile.
The line magic is called "typecheck" to avoid namespace conflict with the mypy
package.
"""

from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    """
    Run the following cell though mypy.
    Any parameters that would normally be passed to the mypy cli
    can be passed on the first line, with the exception of the
    -c flag we use to pass the code from the cell we want to execute
     i.e.
    %%typecheck --ignore-missing-imports
    ...
    ...
    ...
    mypy stdout and stderr will print prior to output of cell. If there are no conflicts,
    nothing will be printed by mypy.
    """

    from IPython import get_ipython
    from mypy import api

    # inserting a newline at the beginning of the cell
    # ensures mypy's output matches the the line
    # numbers in jupyter

    cell = '\n' + cell

    mypy_result = api.run(['-c', cell] + line.split())

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)

In [2]:
import torch

# Verify that CUDA is properly configured and you have a GPU available# Verif 
torch.cuda.is_available()

True

In [3]:
%%typecheck --ignore-missing-imports

import itertools
import random
from typing import Any
from typing import List
from typing import Tuple

low: int = -10
high: int = 10
factors: List[Tuple[Any, ...]]  = [x for x in itertools.product(range(low, high), repeat=2)]
random.shuffle(factors)

In [4]:
%%typecheck --ignore-missing-imports

from typing import Any
from typing import List
from typing import Tuple
import torch
from torch.utils import data

class MultiplicationDataset(data.Dataset):
    '''Dataset for multiplication'''
    
    def __init__(self, factors: List[Tuple[Any, ...]]) -> None:
        self.factors: List[Tuple[Any, ...]] = factors
    
    def __len__(self) -> int:
        return len(self.factors)
    
    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        factors: torch.Tensor = torch.Tensor(self.factors[idx])
        return factors, factors[0] * factors[1]
    
    
def get_datasets(factors: List[Tuple[Any, ...]]) -> Tuple[data.Dataset, data.Dataset]:
    divider: int = int(len(factors) * 0.8)
    train_set: data.Dataset = MultiplicationDataset(factors[:divider])
    test_set: data.Dataset = MultiplicationDataset(factors[divider:])
    return train_set, test_set

In [5]:
batch_size: int = 16384
train_set, test_set = get_datasets(factors)

train_loader = data.DataLoader(dataset=train_set, batch_size=batch_size, 
                               shuffle=True, num_workers=4)
test_loader = data.DataLoader(dataset=test_set, batch_size=batch_size,
                              shuffle=False, num_workers=2)

In [6]:
%%typecheck --ignore-missing-imports

import os
import torch

def save_checkpoint(state, filename: str='checkpoint.pth.tar'):
    torch.save(state, filename)


def load_checkpoint(filename: str='checkpoint.pth.tar'):
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        return torch.load(filename)
    else:
        return None

In [7]:
import tensorflow as tf

def log_scalar_summary(writer, tag, value, step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
    writer.add_summary(summary, step)

In [8]:
%%typecheck --ignore-missing-imports

import torch
from torch import nn
from torch import autograd
from torch.utils import data

dtype = torch.cuda.FloatTensor

def check_accuracy(model: nn.Module, loader: data.DataLoader) -> float:
    model.eval()
    num_samples: int = 0
    num_correct: int = 0
    for x, y in loader:
        x_var: torch.Tensor = autograd.Variable(x.type(dtype), requires_grad=True)
        scores: torch.Tensor = model(x_var).view(-1)
        preds: torch.Tensor = torch.round(scores.data.cpu())
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    return float(num_correct) / num_samples

In [9]:
%%typecheck --ignore-missing-imports

import torch
from torch import nn
from torch import autograd
from torch.utils import data

dtype = torch.cuda.FloatTensor

def get_loss(model: nn.Module, loader: data.DataLoader, loss_fn: nn.Module) -> float:
    model.eval()
    total_loss: float = 0
    num_samples: int = 0
    for x, y in loader:
        x_var: torch.Tensor = autograd.Variable(x.type(dtype), volatile=True)
        y_var: torch.Tensor = y.type(dtype)
        # y_var: torch.Tensor = autograd.Variable(y.type(dtype))
        scores: torch.Tensor = model(x_var).view(-1)
        loss: torch.Tensor = loss_fn(scores, y_var)
        total_loss += loss.data[0].item()
        num_samples += y.shape[0]
    return total_loss / num_samples

In [15]:
%%typecheck --ignore-missing-imports

from typing import Iterable
import tensorflow as tf
import torch
from torch import autograd
from torch import nn
from torch import optim

dtype = torch.cuda.FloatTensor

def train(model: nn.Module, 
          loss_fn: nn.Module, 
          optimizer: optim.Optimizer,
          name: str,
          epochs: Iterable[int],
          epochs_per_save: int=20) -> None:
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                     mode='min',
                                                     factor=0.5, 
                                                     patience=10,
                                                     verbose=True)
    writer = tf.summary.FileWriter(name + '/logdir')
    for epoch in epochs:
        print('Starting epoch {}'.format(epoch + 1))
        total_loss: float = 0
        num_samples: int = 0
        model.train()
        for x, y in train_loader: # type: ignore
            x_var: torch.Tensor = autograd.Variable(x.type(dtype))
            y_var: torch.Tensor = autograd.Variable(y.type(dtype))
            scores: torch.Tensor = model(x_var).view(-1)
            loss: torch.Tensor = loss_fn(scores, y_var)
            total_loss += loss.data[0].item()
            num_samples += y.shape[0]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        training_loss = total_loss / num_samples
        print('training loss:', training_loss)
        log_scalar_summary(writer, 'training loss', training_loss, epoch) # type: ignore
        test_acc = check_accuracy(model, test_loader) # type: ignore
        print('test accuracy:', test_acc)
        log_scalar_summary(writer, 'test accuracy', test_acc, epoch) # type: ignore
        test_loss = get_loss(model, test_loader, loss_fn) # type: ignore
        print('test loss:', test_loss)
        log_scalar_summary(writer, 'test loss', test_loss, epoch) # type: ignore
        
        if scheduler is not None:
            scheduler.step(training_loss)

        if (epoch + 1) % epochs_per_save == 0:
            save_checkpoint({ # type: ignore
                'epoch': epoch,
                'name': name,
                'state_dict': model.state_dict(),
                'acc': test_acc,
                'optimizer' : optimizer.state_dict(),
            }, '{}/{}-{}.pth.tar'.format(name, name, test_acc))

In [32]:
%%typecheck --ignore-missing-imports

import torch
from torch import autograd
from torch import nn

dtype = torch.cuda.FloatTensor

def get_pred(model: nn.Module, x: torch.Tensor) -> torch.Tensor:
    model.eval()
    x_var: torch.Tensor = autograd.Variable(x.type(dtype))
    scores: torch.Tensor = model(x_var).view(-1)
    preds: torch.Tensor = torch.round(scores.data.cpu())
    return preds

In [16]:
%%typecheck --ignore-missing-imports

from typing import List, Sequence
import torch
from torch import nn
from torchvision import models

class Pass(nn.Module):
    def forward(self, x):
        return x

dtype = torch.cuda.FloatTensor

def make_fc(layer_sizes: Sequence[int]=(2, 128, 128, 1),
            dropouts: Sequence[bool]=(False, False, False),
            relus: Sequence[bool]=(True, True, False),
            batchnorms: Sequence[bool]=(True, True, False)) -> nn.Module:
    sequential_layers: List[nn.Module] = []
    # Pair up consecutive layer sizes.
    # For ex, layer_sizes = [a, b, c] -> [(a, b), (b, c)]
    for (layer_size_in, layer_size_out), dropout, relu, batchnorm in zip(
            zip(layer_sizes[:-1], layer_sizes[1:]), dropouts, relus, batchnorms):
        sequential_layers.append(nn.Linear(layer_size_in, layer_size_out))
        if batchnorm:
            sequential_layers.append(nn.BatchNorm1d(layer_size_out))
        if relu:
            sequential_layers.append(nn.ReLU())
        if dropout:
            sequential_layers.append(nn.Dropout())
    model: nn.Module = nn.Sequential(*sequential_layers).type(dtype)
    return model

In [17]:
%%typecheck --ignore-missing-imports

import numpy as np
import torch
from torch import autograd
from torch import nn

dtype = torch.cuda.FloatTensor

model: nn.Module = make_fc() # type: ignore
    
x_var: torch.Tensor = autograd.Variable(torch.randn(2, 2).type(dtype))
ans: torch.Tensor = model(x_var).view(-1)

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([2, ]))

True

In [18]:
loss_fn = nn.MSELoss(reduction='sum').type(dtype)

In [19]:
optimizer = optim.SGD(model.parameters(), lr=5e-5, momentum=0, weight_decay=1e-5)
train(model, loss_fn, optimizer, name='fc3dropout', epochs=range(0, 2000), epochs_per_save=50)

Starting epoch 1
training loss: 1068.2560546875


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


test accuracy: 0.025
test loss: 1242.43759765625
Starting epoch 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss: 971.18349609375
test accuracy: 0.05
test loss: 858.91962890625
Starting epoch 3
training loss: 761.039794921875
test accuracy: 0.025
test loss: 871.308984375
Starting epoch 4
training loss: 463.433642578125
test accuracy: 0.0375
test loss: 610.9353515625
Starting epoch 5
training loss: 228.0179443359375
test accuracy: 0.0
test loss: 8805.275
Starting epoch 6
training loss: 1023.9619140625
test accuracy: 0.025
test loss: 4292.47578125
Starting epoch 7
training loss: 715.87314453125
test accuracy: 0.025
test loss: 1083.95322265625
Starting epoch 8
training loss: 295.384423828125
test accuracy: 0.0625
test loss: 3476.38515625
Starting epoch 9
training loss: 136.48328857421876
test accuracy: 0.025
test loss: 2588.561328125
Starting epoch 10
training loss: 435.17275390625
test accuracy: 0.0125
test loss: 4562.86796875
Starting epoch 11
training loss: 408.1234375
test accuracy: 0.025
test loss: 848.69765625
Starting epoch 12
training loss: 223.1278076171875
test accuracy: 0.01

...

test accuracy: 0.8625
test loss: 0.24805617332458496
Starting epoch 1636
training loss: 0.045603394508361816
test accuracy: 0.8625
test loss: 0.24827492237091064
Starting epoch 1637
training loss: 0.04556092321872711
test accuracy: 0.8625
test loss: 0.24841160774230958
Starting epoch 1638
training loss: 0.04552755355834961
test accuracy: 0.8625
test loss: 0.24917702674865722
Starting epoch 1639
training loss: 0.04549539089202881
test accuracy: 0.8625
test loss: 0.2473311424255371
Starting epoch 1640
training loss: 0.04546511173248291
test accuracy: 0.8625
test loss: 0.24781200885772706
Starting epoch 1641
training loss: 0.04543209671974182
test accuracy: 0.8625
test loss: 0.24848630428314208
Starting epoch 1642
training loss: 0.04539755582809448
test accuracy: 0.8625
test loss: 0.24841411113739015
Starting epoch 1643
training loss: 0.04536543488502502
test accuracy: 0.8625
test loss: 0.247117018699646
Starting epoch 1644
training loss: 0.04533750116825104
test accuracy: 0.8625
test los

test accuracy: 0.8625
test loss: 0.24197447299957275
Starting epoch 1712
training loss: 0.043227067589759825
test accuracy: 0.8625
test loss: 0.24270565509796144
Starting epoch 1713
training loss: 0.04319847524166107
test accuracy: 0.8625
test loss: 0.24272456169128417
Starting epoch 1714
training loss: 0.04316179752349854
test accuracy: 0.8625
test loss: 0.2429023027420044
Starting epoch 1715
training loss: 0.043131425976753235
test accuracy: 0.8625
test loss: 0.24193243980407714
Starting epoch 1716
training loss: 0.043115609884262086
test accuracy: 0.8625
test loss: 0.24217684268951417
Starting epoch 1717
training loss: 0.043081796169281004
test accuracy: 0.8625
test loss: 0.24225320816040039
Starting epoch 1718
training loss: 0.04304913878440857
test accuracy: 0.8625
test loss: 0.24254193305969238
Starting epoch 1719
training loss: 0.04302340447902679
test accuracy: 0.8625
test loss: 0.24146203994750975
Starting epoch 1720
training loss: 0.04300006628036499
test accuracy: 0.8625
tes

test accuracy: 0.8625
test loss: 0.2382587194442749
Starting epoch 1788
training loss: 0.04075719118118286
test accuracy: 0.8625
test loss: 0.23805794715881348
Starting epoch 1789
training loss: 0.040718796849250796
test accuracy: 0.8625
test loss: 0.23803117275238037
Starting epoch 1790
training loss: 0.04069423079490662
test accuracy: 0.8625
test loss: 0.23687028884887695
Starting epoch 1791
training loss: 0.040665853023529056
test accuracy: 0.8625
test loss: 0.23772947788238524
Starting epoch 1792
training loss: 0.040634354948997496
test accuracy: 0.8625
test loss: 0.23773868083953859
Starting epoch 1793
training loss: 0.040596804022789
test accuracy: 0.8625
test loss: 0.23794476985931395
Starting epoch 1794
training loss: 0.04057077169418335
test accuracy: 0.8625
test loss: 0.23697001934051515
Starting epoch 1795
training loss: 0.040543511509895325
test accuracy: 0.8625
test loss: 0.2370211362838745
Starting epoch 1796
training loss: 0.04050824642181396
test accuracy: 0.8625
test l

test accuracy: 0.8625
test loss: 0.23370201587677003
Starting epoch 1864
training loss: 0.03842689394950867
test accuracy: 0.8625
test loss: 0.23362352848052978
Starting epoch 1865
training loss: 0.03839724361896515
test accuracy: 0.8625
test loss: 0.2321463108062744
Starting epoch 1866
training loss: 0.038372156023979184
test accuracy: 0.8625
test loss: 0.23302903175354003
Starting epoch 1867
training loss: 0.038343149423599246
test accuracy: 0.8625
test loss: 0.23291456699371338
Starting epoch 1868
training loss: 0.03830886781215668
test accuracy: 0.8625
test loss: 0.23312249183654785
Starting epoch 1869
training loss: 0.03827868103981018
test accuracy: 0.8625
test loss: 0.23320417404174804
Starting epoch 1870
training loss: 0.03825724720954895
test accuracy: 0.8625
test loss: 0.23226702213287354
Starting epoch 1871
training loss: 0.03822916448116302
test accuracy: 0.8625
test loss: 0.23257241249084473
Starting epoch 1872
training loss: 0.038195517659187314
test accuracy: 0.8625
test

test accuracy: 0.8625
test loss: 0.22810451984405516
Starting epoch 1940
training loss: 0.03628326952457428
test accuracy: 0.8625
test loss: 0.2284034013748169
Starting epoch 1941
training loss: 0.03625498712062836
test accuracy: 0.8625
test loss: 0.22883501052856445
Starting epoch 1942
training loss: 0.03623523712158203
test accuracy: 0.8625
test loss: 0.22720074653625488
Starting epoch 1943
training loss: 0.03621042966842651
test accuracy: 0.8625
test loss: 0.22783749103546141
Starting epoch 1944
training loss: 0.036178994178771975
test accuracy: 0.8625
test loss: 0.22787873744964598
Starting epoch 1945
training loss: 0.03614960908889771
test accuracy: 0.8625
test loss: 0.2284008264541626
Starting epoch 1946
training loss: 0.03612450361251831
test accuracy: 0.8625
test loss: 0.22841265201568603
Starting epoch 1947
training loss: 0.03610149621963501
test accuracy: 0.8625
test loss: 0.22705557346343994
Starting epoch 1948
training loss: 0.03607639670372009
test accuracy: 0.8625
test lo

In [20]:
optimizer = optim.SGD(model.parameters(), lr=1.25e-5, momentum=0, weight_decay=1e-5)
train(model, loss_fn, optimizer, name='fc3dropout', epochs=range(2000, 4000), epochs_per_save=50)

Starting epoch 2001
training loss: 0.03471989631652832


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


test accuracy: 0.8625
test loss: 0.2237165689468384
Starting epoch 2002


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss: 0.03469696640968323
test accuracy: 0.8625
test loss: 0.22360796928405763
Starting epoch 2003
training loss: 0.034670135378837584
test accuracy: 0.8625
test loss: 0.22377915382385255
Starting epoch 2004
training loss: 0.03464767336845398
test accuracy: 0.8625
test loss: 0.22240784168243408
Starting epoch 2005
training loss: 0.03463041484355926
test accuracy: 0.8625
test loss: 0.2231276512145996
Starting epoch 2006
training loss: 0.034604400396347046
test accuracy: 0.8625
test loss: 0.22306089401245116
Starting epoch 2007
training loss: 0.034577357769012454
test accuracy: 0.8625
test loss: 0.22303290367126466
Starting epoch 2008
training loss: 0.03455377221107483
test accuracy: 0.8625
test loss: 0.22330992221832274
Starting epoch 2009
training loss: 0.03453080356121063
test accuracy: 0.8625
test loss: 0.2221832275390625
Starting epoch 2010
training loss: 0.03451170325279236
test accuracy: 0.8625
test loss: 0.22242491245269774
Starting epoch 2011
training loss: 0.0344878047

...

test accuracy: 0.9375
test loss: 0.15749516487121581
Starting epoch 3684
training loss: 0.014635758101940155
test accuracy: 0.95
test loss: 0.15758339166641236
Starting epoch 3685
training loss: 0.014632675051689147
test accuracy: 0.9375
test loss: 0.1577164888381958
Starting epoch 3686
training loss: 0.014623647928237915
test accuracy: 0.95
test loss: 0.15756568908691407
Starting epoch 3687
training loss: 0.014617243409156799
test accuracy: 0.9375
test loss: 0.1580023407936096
Starting epoch 3688
training loss: 0.014611625671386718
test accuracy: 0.95
test loss: 0.15778698921203613
Starting epoch 3689
training loss: 0.014607414603233337
test accuracy: 0.9375
test loss: 0.15797033309936523
Starting epoch 3690
training loss: 0.014603802561759948
test accuracy: 0.95
test loss: 0.15678718090057372
Starting epoch 3691
training loss: 0.014607706665992736
test accuracy: 0.9375
test loss: 0.1573728322982788
Starting epoch 3692
training loss: 0.014596876502037049
test accuracy: 0.95
test loss:

test accuracy: 0.95
test loss: 0.15535480976104737
Starting epoch 3761
training loss: 0.014204040169715881
test accuracy: 0.95
test loss: 0.15602481365203857
Starting epoch 3762
training loss: 0.01419895440340042
test accuracy: 0.95
test loss: 0.15587227344512938
Starting epoch 3763
training loss: 0.01419139951467514
test accuracy: 0.95
test loss: 0.156042742729187
Starting epoch 3764
training loss: 0.01418474018573761
test accuracy: 0.95
test loss: 0.15603973865509033
Starting epoch 3765
training loss: 0.014180028438568115
test accuracy: 0.95
test loss: 0.1560977578163147
Starting epoch 3766
training loss: 0.014176522195339204
test accuracy: 0.95
test loss: 0.15593900680541992
Starting epoch 3767
training loss: 0.014168477058410645
test accuracy: 0.95
test loss: 0.15616246461868286
Starting epoch 3768
training loss: 0.014162790775299073
test accuracy: 0.95
test loss: 0.15604125261306762
Starting epoch 3769
training loss: 0.014159071445465087
test accuracy: 0.95
test loss: 0.1564305782

test accuracy: 0.95
test loss: 0.15492323637008668
Starting epoch 3838
training loss: 0.013827098906040192
test accuracy: 0.95
test loss: 0.15367690324783326
Starting epoch 3839
training loss: 0.013830558955669403
test accuracy: 0.95
test loss: 0.15413963794708252
Starting epoch 3840
training loss: 0.013821063935756684
test accuracy: 0.95
test loss: 0.15406233072280884
Starting epoch 3841
training loss: 0.01381586492061615
test accuracy: 0.95
test loss: 0.1541820764541626
Starting epoch 3842
training loss: 0.013809849321842194
test accuracy: 0.95
test loss: 0.15409001111984252
Starting epoch 3843
training loss: 0.013802267611026764
test accuracy: 0.95
test loss: 0.15451234579086304
Starting epoch 3844
training loss: 0.01379946619272232
test accuracy: 0.95
test loss: 0.15416934490203857
Starting epoch 3845
training loss: 0.013794335722923278
test accuracy: 0.95
test loss: 0.15462677478790282
Starting epoch 3846
training loss: 0.01379188448190689
test accuracy: 0.95
test loss: 0.15417683

test accuracy: 0.95
test loss: 0.15309947729110718
Starting epoch 3915
training loss: 0.013510939478874207
test accuracy: 0.95
test loss: 0.15325876474380493
Starting epoch 3916
training loss: 0.013503518700599671
test accuracy: 0.95
test loss: 0.15301579236984253
Starting epoch 3917
training loss: 0.013501575589179993
test accuracy: 0.95
test loss: 0.15340055227279664
Starting epoch 3918
training loss: 0.013500657677650452
test accuracy: 0.95
test loss: 0.1521235942840576
Starting epoch 3919
training loss: 0.013505777716636658
test accuracy: 0.95
test loss: 0.1525750994682312
Starting epoch 3920
training loss: 0.013499829173088073
test accuracy: 0.95
test loss: 0.1524984359741211
Starting epoch 3921
training loss: 0.013492527604103088
test accuracy: 0.95
test loss: 0.15265640020370483
Starting epoch 3922
training loss: 0.013486392796039581
test accuracy: 0.95
test loss: 0.1525905728340149
Starting epoch 3923
training loss: 0.013484986126422882
test accuracy: 0.95
test loss: 0.15295065

test accuracy: 0.95
test loss: 0.15181740522384643
Starting epoch 3992
training loss: 0.013234394788742065
test accuracy: 0.95
test loss: 0.1514374852180481
Starting epoch 3993
training loss: 0.013229601085186005
test accuracy: 0.95
test loss: 0.1516004204750061
Starting epoch 3994
training loss: 0.013225437700748443
test accuracy: 0.95
test loss: 0.15144439935684204
Starting epoch 3995
training loss: 0.013220933079719544
test accuracy: 0.95
test loss: 0.1519161581993103
Starting epoch 3996
training loss: 0.013224023580551147
test accuracy: 0.95
test loss: 0.15152214765548705
Starting epoch 3997
training loss: 0.013217826187610627
test accuracy: 0.95
test loss: 0.15164587497711182
Starting epoch 3998
training loss: 0.013214072585105896
test accuracy: 0.95
test loss: 0.1514817953109741
Starting epoch 3999
training loss: 0.013205556571483612
test accuracy: 0.95
test loss: 0.15188677310943605
Starting epoch 4000
training loss: 0.013202536106109618
test accuracy: 0.95
test loss: 0.15149884

In [21]:
optimizer = optim.SGD(model.parameters(), lr=1.25e-5, momentum=0, weight_decay=1e-5)
train(model, loss_fn, optimizer, name='fc3dropout', epochs=range(4000, 6000), epochs_per_save=50)

Starting epoch 4001
training loss: 0.013198490440845489


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


test accuracy: 0.95
test loss: 0.15163310766220092
Starting epoch 4002


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss: 0.01320059597492218
test accuracy: 0.95
test loss: 0.15134048461914062
Starting epoch 4003
training loss: 0.013191647827625275
test accuracy: 0.95
test loss: 0.1518249988555908
Starting epoch 4004
training loss: 0.013194386661052705
test accuracy: 0.95
test loss: 0.15141996145248413
Starting epoch 4005
training loss: 0.013184930384159087
test accuracy: 0.95
test loss: 0.15186965465545654
Starting epoch 4006
training loss: 0.013187168538570404
test accuracy: 0.95
test loss: 0.15144964456558227
Starting epoch 4007
training loss: 0.013177503645420075
test accuracy: 0.95
test loss: 0.15161747932434083
Starting epoch 4008
training loss: 0.013178060948848724
test accuracy: 0.95
test loss: 0.15137863159179688
Starting epoch 4009
training loss: 0.013167095184326173
test accuracy: 0.95
test loss: 0.15194346904754638
Starting epoch 4010
training loss: 0.013163767755031586
test accuracy: 0.95
test loss: 0.15148152112960817
Starting epoch 4011
training loss: 0.013152986764907837
tes

...

training loss: 0.009609764814376831
test accuracy: 0.95
test loss: 0.14114583730697633
Starting epoch 5620
training loss: 0.009608329087495805
test accuracy: 0.95
test loss: 0.14111430644989015
Starting epoch 5621
training loss: 0.009607044607400894
test accuracy: 0.95
test loss: 0.14108362197875976
Starting epoch 5622
training loss: 0.009605514258146286
test accuracy: 0.95
test loss: 0.14111908674240112
Starting epoch 5623
training loss: 0.009604030847549438
test accuracy: 0.95
test loss: 0.14113757610321045
Starting epoch 5624
training loss: 0.009602567553520203
test accuracy: 0.95
test loss: 0.14104464054107665
Starting epoch 5625
training loss: 0.009601336717605592
test accuracy: 0.95
test loss: 0.14107775688171387
Starting epoch 5626
training loss: 0.00959949940443039
test accuracy: 0.95
test loss: 0.14112504720687866
Starting epoch 5627
training loss: 0.009597937762737273
test accuracy: 0.95
test loss: 0.14113484621047973
Starting epoch 5628
training loss: 0.009596457332372665
te

training loss: 0.009499282389879227
test accuracy: 0.95
test loss: 0.1407931923866272
Starting epoch 5697
training loss: 0.009497630596160888
test accuracy: 0.95
test loss: 0.1407390594482422
Starting epoch 5698
training loss: 0.009496942162513733
test accuracy: 0.95
test loss: 0.1407386541366577
Starting epoch 5699
training loss: 0.009495405852794648
test accuracy: 0.95
test loss: 0.1407619833946228
Starting epoch 5700
training loss: 0.009493760019540786
test accuracy: 0.95
test loss: 0.14076945781707764
Starting epoch 5701
training loss: 0.009492038935422897
test accuracy: 0.95
test loss: 0.14076145887374877
Starting epoch 5702
training loss: 0.00949067622423172
test accuracy: 0.95
test loss: 0.14075210094451904
Starting epoch 5703
training loss: 0.009489486366510392
test accuracy: 0.95
test loss: 0.140688157081604
Starting epoch 5704
training loss: 0.009488561004400254
test accuracy: 0.95
test loss: 0.14070165157318115
Starting epoch 5705
training loss: 0.009486982226371765
test acc

training loss: 0.00939219370484352
test accuracy: 0.95
test loss: 0.14034297466278076
Starting epoch 5774
training loss: 0.009391088783740998
test accuracy: 0.95
test loss: 0.14032247066497802
Starting epoch 5775
training loss: 0.009389755129814149
test accuracy: 0.95
test loss: 0.14029675722122192
Starting epoch 5776
training loss: 0.009388570487499238
test accuracy: 0.95
test loss: 0.14027273654937744
Starting epoch 5777
training loss: 0.009387186914682388
test accuracy: 0.95
test loss: 0.14027791023254393
Starting epoch 5778
training loss: 0.009385316073894501
test accuracy: 0.95
test loss: 0.1403173565864563
Starting epoch 5779
training loss: 0.009383903443813324
test accuracy: 0.95
test loss: 0.140289831161499
Starting epoch 5780
training loss: 0.009382674843072892
test accuracy: 0.95
test loss: 0.14026176929473877
Starting epoch 5781
training loss: 0.009381815791130066
test accuracy: 0.95
test loss: 0.14021060466766358
Starting epoch 5782
training loss: 0.009380347281694412
test 

training loss: 0.009287140518426894
test accuracy: 0.95
test loss: 0.13985921144485475
Starting epoch 5851
training loss: 0.009285424649715424
test accuracy: 0.95
test loss: 0.13986315727233886
Starting epoch 5852
training loss: 0.00928439199924469
test accuracy: 0.95
test loss: 0.1398390531539917
Starting epoch 5853
training loss: 0.00928337424993515
test accuracy: 0.95
test loss: 0.1397736430168152
Starting epoch 5854
training loss: 0.009281723201274872
test accuracy: 0.95
test loss: 0.1398058295249939
Starting epoch 5855
training loss: 0.009280404448509217
test accuracy: 0.95
test loss: 0.1398139238357544
Starting epoch 5856
training loss: 0.00927886888384819
test accuracy: 0.95
test loss: 0.13983243703842163
Starting epoch 5857
training loss: 0.00927734151482582
test accuracy: 0.95
test loss: 0.13983240127563476
Starting epoch 5858
training loss: 0.00927652046084404
test accuracy: 0.95
test loss: 0.13981139659881592
Starting epoch 5859
training loss: 0.009274933487176895
test accur

training loss: 0.009184157848358155
test accuracy: 0.95
test loss: 0.13935939073562623
Starting epoch 5928
training loss: 0.00918281227350235
test accuracy: 0.95
test loss: 0.13933494091033935
Starting epoch 5929
training loss: 0.009181399643421174
test accuracy: 0.95
test loss: 0.13934972286224365
Starting epoch 5930
training loss: 0.009180375188589097
test accuracy: 0.95
test loss: 0.13931978940963746
Starting epoch 5931
training loss: 0.009178911149501801
test accuracy: 0.95
test loss: 0.13931766748428345
Starting epoch 5932
training loss: 0.009177744388580322
test accuracy: 0.95
test loss: 0.13926485776901246
Starting epoch 5933
training loss: 0.00917646884918213
test accuracy: 0.95
test loss: 0.1393013596534729
Starting epoch 5934
training loss: 0.00917491614818573
test accuracy: 0.95
test loss: 0.13929734230041504
Starting epoch 5935
training loss: 0.00917351245880127
test accuracy: 0.95
test loss: 0.13932181596755983
Starting epoch 5936
training loss: 0.00917242169380188
test ac

In [34]:
get_pred(model, torch.tensor([[7, 9], [-2, 4]]))

tensor([63., -8.])